In [56]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

auth.authenticate_user()
project_id = "mindful-carport-440904-q3"
client = bigquery.Client(project=project_id)


Coba dengan Filing_Date

In [93]:
# Your supervisor wants to see the cumulative number of patents related to cancer on an annual basis in the ministry database with condition :
# - the patent was registered in 2012 to 2023
# - the patent contains the keyword "data mining"
# Display the cumulative number of patents sorted by largest year and smallest month.4

# 1. Buat CTE utk tabel sementara, 1 tabel untuk ubah tipe data dari string ke date, 1 tabel untuk ekstrak tahun dan bulan
# 2. Dari kolom smentara yang sudah ada tahun dan bulannya:
      # - Filter berdasarkan tahun, harus diantara 2012 dan 2023
      # - Patent_title diubah semua ke lower case karna tidak bisa pakai ilike, supaya semua yang ada data miningnya terambil
# 3. Munculkan kolom tahun, bulan dan hitung barisnya
# 4. Grouping berdasarkan tahun dan bulan
# 5. Diurutkan untuk tahun dari besar ke kecil, untuk bulan dari kecil ke besar


query = """
WITH newPublications as(
  SELECT
  *,
  PARSE_DATE('%Y%m%d', Filing_Date) as New_Filling_Date,
FROM
  `patents-public-data.uspto_oce_cancer.publications`
),
yearMonthColumns as (
  SELECT *,
    EXTRACT(YEAR FROM New_Filling_Date) as year,
    EXTRACT(MONTH FROM New_Filling_Date) as month
  FROM newPublications
)

SELECT
      year,
      month,
      count(*) as total_patent
from yearMonthColumns
WHERE
  year BETWEEN 2012 AND 2023
  AND
  LOWER(Patent_Title) LIKE '%data mining%'
GROUP BY year, month
ORDER BY year desc, month asc
;
"""

query_job = client.query(query).to_dataframe()
query_job

,year,month,total_patent
0,2014,12,2
1,2012,4,2


Coba dengan Grant_or_Publications_Date

In [92]:
query = """
WITH newPublications as(
  SELECT
  *,
  PARSE_DATE('%Y%m%d', Grant_or_Publication_Date) as Grant_Date,
FROM
  `patents-public-data.uspto_oce_cancer.publications`
),
yearMonthColumns as (
  SELECT *,
    EXTRACT(YEAR FROM Grant_Date) as year,
    EXTRACT(MONTH FROM Grant_Date) as month
  FROM newPublications
)

SELECT
      year,
      month,
      count(*) as total_patent
from yearMonthColumns
WHERE
  year BETWEEN 2012 AND 2023
  AND
  LOWER(Patent_Title) LIKE '%data mining%'
GROUP BY year, month
ORDER BY year desc, month asc
;
"""

query_job = client.query(query).to_dataframe()
query_job

,year,month,total_patent
0,2015,2,1
1,2015,4,1
2,2015,10,1
3,2014,1,2
4,2014,3,1
5,2012,7,2
6,2012,8,1
7,2012,10,1
